<a href="https://colab.research.google.com/github/CodeMonkey01/DataMiningI/blob/main/ANN/Option_A/ANN_with_BERT_add_BERT_seperator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANN with BERT
In this notebook I tried to solve the classification model with an ANN based on pretrained BERT layers.

## TODO:
- [ ] Stemming
- [ ] Remove Stop Words
- [x] BERT Seperator

## Results
accuracy: 0.9297 - precision: 0.9260 - recall: 0.9341

## Model
Link to the model: https://drive.google.com/file/d/1-DcCa5FowzP-bHRPYxY7t0MAwo5gnBjN/view?usp=sharing, https://drive.google.com/file/d/1-IqI6Xf3s4oU5Eer6D46Hdn3405qw8uD/view?usp=sharing 

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd '/content/drive/MyDrive/'

    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)
except ImportError as e:
    pass

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive
Thu May 19 08:01:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------

In [2]:
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install transformers

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [4]:
df_raw = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset.txt')
df_raw.describe()

,text,humor
count,200000,200000
unique,200000,2
top,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
freq,1,100000


In [5]:
# todo --> take random sample to speed up training
df = df_raw.sample(5000)

# Check for imbalance
The dataset is equally balanced. Therefore, we do not need to rebalance the dataset.

In [6]:
df["humor"].value_counts()

True     2508
False    2492
Name: humor, dtype: int64

# Preprocessing

In [7]:
# Transform class from Boolean to integer value
df['class']=df['humor'].apply(lambda x: 1 if x==True else 0)

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

MAX_LEN = 128
#pad_to_max_length=True,
df['bert_preprocessed']=df['text'].apply(lambda x: " ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(x, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))))

In [9]:
df.head()

,text,humor,class,bert_preprocessed
67815,10 good reasons to get a flu shot,False,0,[CLS] 10 good reasons to get a flu shot [SEP]
149041,"I leave the interview room as i entered it, ka...",True,1,[CLS] i leave the interview room as i entered ...
178343,I wish i was ugly for just one single day.. be...,True,1,[CLS] i wish i was ugly for just one single da...
159016,Do competitive origami artists ever fold under...,True,1,[CLS] do competitive or ##iga ##mi artists eve...
112070,Litter from indonesia -- rubble in paradise: c...,False,0,[CLS] litter from indonesia - - rubble in para...


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['bert_preprocessed'],df['class'], test_size=0.4)

# BERT

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

## Test embedding
Test word embedding from pretrained BERT model with a real sentence from dataset.

In [13]:
test_sentence = df["bert_preprocessed"][11704]
print("Test sentence:")
print(test_sentence)
print("Test sentence (word embedding):")
print(get_sentence_embeding([test_sentence]))

KeyError: ignored

# Build model

In [14]:
def build_model() -> tf.keras.Model:
    # Bert layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

    # Softmax is shit for this binary classification problem. Event though the literature suggest this activation function for binary class problem
    #l = tf.keras.layers.Dense(1, activation='softmax', name="output")(l) 

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[text_input], outputs = [l])

    #model.summary()

    return model

In [ ]:
import numpy as np
EPOCHS = 5

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

#model.compile(optimizer='adam',
#              loss='binary_crossentropy',
#              metrics=METRICS)

lrs = np.arange(1e-3, 1e-2, 0.0002)
print(f"Combinations: {len(lrs)}")
for lr in lrs:
    model: tf.keras.Model = build_model()

    print(f"Try adam learning rate of: {lr}")

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=METRICS)
    
    model.fit(X_train, y_train, epochs=EPOCHS)

Combinations: 45
Try adam learning rate of: 0.001
Epoch 1/5
94/94 [==============================] - 38s 306ms/step - loss: 0.6259 - accuracy: 0.6470 - precision: 0.6365 - recall: 0.6943
Epoch 2/5
94/94 [==============================] - 29s 311ms/step - loss: 0.5231 - accuracy: 0.7667 - precision: 0.7488 - recall: 0.8064
Epoch 3/5
94/94 [==============================] - 29s 310ms/step - loss: 0.4701 - accuracy: 0.7987 - precision: 0.7776 - recall: 0.8395
Epoch 4/5
94/94 [==============================] - 29s 308ms/step - loss: 0.4403 - accuracy: 0.8157 - precision: 0.8032 - recall: 0.8389
Epoch 5/5
94/94 [==============================] - 29s 309ms/step - loss: 0.4143 - accuracy: 0.8323 - precision: 0.8170 - recall: 0.8588
Try adam learning rate of: 0.0012000000000000001
Epoch 1/5
94/94 [==============================] - 31s 309ms/step - loss: 0.6285 - accuracy: 0.7387 - precision: 0.7253 - recall: 0.7729
Epoch 2/5
94/94 [==============================] - 29s 310ms/step - loss: 0.520

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
#model.save_weights("/content/drive/MyDrive/Data Mining/bert_seperator/weights.md5")

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

# Own test

In [ ]:
jokes = [
    'What’s the best thing about Switzerland? I don’t know, but the flag is a big plus.',
    'I study Business Informatics at the University of Mannheim!',
    'I invented a new word! Plagiarism!',
    'Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them.',
    'My name is Elias.'
]

jokes_processed = [" ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(i, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))) for i in jokes]

model.predict(jokes_processed)